In [1]:
from math import sqrt
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import random
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, r2_score
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization
from keras.layers import LSTM
from keras.layers import Flatten
from keras.layers import TimeDistributed
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
import datetime
import seaborn as sns
from scipy import stats

In [2]:
def scale(dataset,col_name, output_array=False):
    """Standard Z score scaler"""
    array = dataset[col_name].to_numpy()
    mean = np.mean(array)
    std = np.std(array)
    scaled = (array-mean)/std
    if output_array:
        return scaled
    else:
        return scaled, mean, std 

In [3]:
def series_to_supervised(data, n_in=1, n_out=1, variable_num = 0,  dropnan=True):
    """
    Frame a time series as a supervised learning dataset.
    Arguments:
    data: Sequence of observations as a list or NumPy array.
    n_in: Number of lag observations as input (X).
    n_out: Number of observations as output (y).
    dropnan: Boolean whether or not to drop rows with NaN values.
    Returns:
    Pandas DataFrame of series framed for supervised learning.
    """
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names+=[('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df[variable_num].shift(-i))
        if i == 0:
            names.append('var1(t)')
        else:
            names.append('var%d(t+%d)' % (1, i))
    
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg, names

In [4]:
def random_date(array, lockdown=False):
    """ Choose a random date within lockdown or out of lockdown."""
    if lockdown:
        return np.random.choice(array[(array > pd.to_datetime('2020-03-24')) & (array < pd.to_datetime('2020-06-23')) ])
    else:
        #ensure a years worth of hourly data is available for training.
        array = pd.to_datetime(array[(24*365):])
        
        # 2020-01-01 ensures only non-lockdown entries will be considered
        return np.random.choice(array[array < pd.to_datetime('2020-01-01')])

In [5]:
def split_by_date( dataset , reframed_data, n_hours, date, days_prior=60):
    
    """ Split reframed data by specific date, days refers to the number 
    of days after the specified date the test set will contain.
    days prior refers to the number of days used to train the model """
    
    #get values from reframed data
    values = reframed_data.values
    
    #convert string into datetime
    date = pd.to_datetime(date)
    
    #create new dataframe
    df = dataset.reset_index().rename(columns = {'index':'date'})
    
    #ensure dates are in datetime format
    df['date'] = pd.to_datetime(df['date'])
    
    #find index for date
    date1_index = df.index[df['date'] == date ][0]
    
    #set train values to be all data upto given date 
    train = values[ date1_index - n_hours - (24*days_prior) : date1_index - n_hours, :]
    
    #remove training dataset so that tests can be done on the remaining datapoints 
    leftover_before = values[ : date1_index - n_hours - (24 * days_prior) , :]
    leftover_after = values[ date1_index - n_hours : , :]

    #get index of second date is 1 month into future
    date2_index = date1_index + (30 * 24)

    #test set two months after specified date
    test = values[ ( date1_index - n_hours ) : (date2_index - n_hours)  + 1 , :]
    
    #get specific dates for test period
    dates = df.date.to_numpy()
    specific_dates = dates[ date1_index  : date2_index+1  ]
    
    if leftover_before.shape[0] > leftover_after.shape[0]:
        
        return train , test, specific_dates, leftover_before
    
    else:
        return train , test, specific_dates, leftover_after

In [6]:
def reshape(data,n_hours, n_seq, n_steps, n_features):
    """ Reshape puts data in apprpriate format for LSTM_CNN usage
    [samples, n_seqs, timesteps, features] """
    n_obs = n_hours * n_features

    #split according to lag columns and output
    data_X, data_y = data[:, :n_obs], data[:, n_obs:]
         
    # reshape input to be 3D [samples, timesteps, features]
    data_X_r = data_X.reshape((data_X.shape[0], n_seq , n_steps, n_features))

    return data_X, data_y , data_X_r 

In [7]:
def inverse_scale(array,mean,std):
    """ Inversion of Standard scaler """
    inverse = (array*std) + mean 
    return inverse

In [8]:
def CNN_LSTM_model(cells, filter_no, filter_size, dropout, n_hours, n_steps, n_features):
    # define model
    model = Sequential()
    model.add(TimeDistributed(Conv1D(filters = filter_no, padding='same', kernel_size=filter_size, activation='relu'), input_shape=(None, n_steps, n_features)))

    filter_no = int ( filter_no / 2 )
    filter_size += 2
    dropout += 0.1
    model.add(TimeDistributed(Conv1D(filters = filter_no, padding='same', kernel_size=filter_size, activation='relu'), input_shape=(None, n_steps, n_features)))
    model.add(TimeDistributed(Conv1D(filters = filter_no, padding='same', kernel_size=filter_size, activation='relu'), input_shape=(None, n_steps, n_features)))
    model.add(TimeDistributed(BatchNormalization()))
    model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
    model.add(TimeDistributed(Dropout(dropout)))

    filter_no = int ( filter_no / 2 )
    filter_size += 2
    dropout += 0.1
    model.add(TimeDistributed(Conv1D(filters = filter_no, padding='same', kernel_size=filter_size, activation='relu'), input_shape=(None, n_steps, n_features)))
    model.add(TimeDistributed(Conv1D(filters = filter_no, padding='same', kernel_size=filter_size, activation='relu'), input_shape=(None, n_steps, n_features)))
    model.add(TimeDistributed(BatchNormalization()))
    model.add(TimeDistributed(MaxPooling1D(pool_size=1)))
    model.add(TimeDistributed(Dropout(dropout)))
    
    model.add(TimeDistributed(Flatten()))
    model.add(LSTM(cells, activation = 'relu',input_shape=(train_X_r.shape[1], train_X_r.shape[2])))
    model.add(Dense(n_hours, activation = 'relu'))
    model.add(Dense(n_hours_out))
    model.compile(loss='mse', optimizer='adam')
    
    return model

In [9]:
def hour_predictor(test_r, index , timesteps):
    """ Rolling one hour prediction: uses previous 
    1 hour predictions to predict 1 hour ahead """
    
    #starting sequence used to get first prediction. 
    start = test_r[index].reshape(1,n_hours,n_features)
    
    #empty array to store results 
    results = []
    
    #get first prediction
    result = model.predict(start)[0][0]
    
    #record it 
    results.append(result)
    
    for i in range (1,timesteps):

        #shift window one time step
        start = np.delete(start[0],0,0)
        next_ts = test_r[index+i][2]
        start = np.append(start,next_ts).reshape(1,n_hours,n_features)

        #add prediction
        start[0][n_hours-1][0] = result

        #make new prediction
        result = model.predict(start)[0][0]

        results.append(result)
        
    return np.array(results)

In [10]:
def data_prep(site, period, variables ):

        dataset = pd.read_csv('./'+ site + '_combined.csv', header=0, index_col=0)

        dataset.index = pd.DatetimeIndex(dataset.index,freq='h')

        #get all dates
        all_dates = pd.to_datetime(dataset.index.to_numpy())

        #scale continuous variables --> ['temp', 'ws', 'wd', 'Volume']
        continuous_vars = ['temp', 'ws', 'wd', 'Volume']
        for i in continuous_vars:
            dataset[i] = scale(dataset,i,output_array=True)

        # scale NO2 keep mean and std for inverse scaling
        no2, mean, std, = scale(dataset,'NO2',output_array=False)
        dataset['NO2'] = no2
        
        # chose specific variables
        dataset_copy = dataset[variables]

        hours_before = 24
        hours_after = 1

        days_before = 1
        days_after = 1

        # # specify the number of lag hours
        n_hours = hours_before * days_before # lag hours for each feature (t-x)
        n_features = len(variables) # number of features 
        n_hours_out = hours_after * days_after # lenghth of output forecast (<1 == <t+1)

        #date1 = period

        #convert values
        values = dataset_copy.values.astype('float32') 

        # frame as supervised learning
        reframed, names = series_to_supervised(values, n_hours, n_hours_out )


        train , test,  dates , leftover = split_by_date(dataset_copy, reframed, n_hours, period, 365)

        n_seq = 3
        n_steps = int( n_hours / n_seq )

        train_X, train_y, train_X_r = reshape(train, n_hours, n_seq, n_steps, n_features)
        test_X, test_y, test_X_r = reshape(test, n_hours, n_seq, n_steps, n_features)
        
        return train_X_r, train_y, test_X_r, test_y, n_hours ,n_steps, n_hours_out,n_features, dates, mean , std




In [11]:
def evaluate_model(train_X_r, train_y, test_X_r, test_y,n_hours,n_steps, n_hours_out, n_features,  dates, mean , std):
    
    model = CNN_LSTM_model(24, 32, 3, 0, n_hours,n_steps,n_features)

    # fit network
    history = model.fit(train_X_r, train_y, epochs=100, batch_size=128, validation_split = 0.2 , verbose=0, shuffle=False)

    # get scaled prediction and observed values to calculate RMSE
    predicted = model.predict(test_X_r)[::n_hours_out]
    observed = test_y[::n_hours_out]

    # rescale for plotting
    predicted_rescaled = inverse_scale(predicted.flatten(), mean, std)[:dates.shape[0]]
    observed_rescaled = inverse_scale(observed.flatten(), mean, std)[:dates.shape[0]]

    
    return observed_rescaled, predicted_rescaled


In [12]:
def plot_trace (observed_rescaled, predicted_rescaled,dates,plot_days,colour, var_label):
    MAPE = mean_absolute_percentage_error(observed_rescaled,predicted_rescaled)
    corr, p = stats.pearsonr(observed_rescaled,predicted_rescaled) 
    RMSE = mean_squared_error(observed_rescaled, predicted_rescaled,squared = False)
    plt.plot(dates[:24*plot_days],observed_rescaled[:24*plot_days],color = 'r',label='Observed')
    plt.plot(dates[:24*plot_days],predicted_rescaled[:24*plot_days],color = colour,label=var_label)
    plt.xticks(rotation=45,fontsize=25)
    #plt.title('RMSE: %.3f, MAPE: %.3f, corr: %.3f (p=%.1f)' % (round(RMSE, 3),round(MAPE, 3),round(corr, 3), round(p,3)),fontsize=25) 
    plt.legend(prop={'size': 25})
    plt.yticks(fontsize=25)
    plt.ylabel('NO${_2}$ (${\mu g/m^{3}}$)',fontsize=25)

In [13]:
VARIABLES = [['NO2'],['NO2','Volume'],['NO2','ws'],
             ['NO2','ws' ,'temp','wd_1', 'wd_2', 'wd_3', 'wd_4'],
             ['NO2','Volume' ,'ws' ,'temp','wd_1', 'wd_2', 'wd_3', 'wd_4'],
             ['NO2', 'ws' ,'temp','wd_1', 'wd_2', 'wd_3', 'wd_4', 'Volume','Mon', 'Tue', 'Wed',
             'Thu', 'Fri', 'Sat', 'Sun']]

In [14]:
train_X_r, train_y, test_X_r, test_y, n_hours,n_steps,n_hours_out,n_features, dates_nld, mean , std  = data_prep('MCC','2020-01-23',['NO2'])
observed_p_NO2, predicted_p_NO2 = evaluate_model(train_X_r, train_y, test_X_r, test_y,n_hours,n_steps, n_hours_out,n_features, dates_nld, mean , std)

train_X_r, train_y, test_X_r, test_y, n_hours,n_steps,n_hours_out,n_features, dates_ld, mean , std  = data_prep('MCC','2020-03-23', ['NO2'])
observed_p_NO2LD, predicted_p_NO2LD = evaluate_model(train_X_r, train_y, test_X_r, test_y,n_hours,n_steps, n_hours_out,n_features, dates_ld, mean , std)

train_X_r, train_y, test_X_r, test_y, n_hours,n_steps,n_hours_out,n_features, dates, mean , std  = data_prep('MCC','2020-01-23', ['NO2','Volume'])
observed_p_NO2vol, predicted_p_NO2vol = evaluate_model(train_X_r, train_y, test_X_r, test_y,n_hours,n_steps, n_hours_out,n_features, dates, mean , std)

train_X_r, train_y, test_X_r, test_y, n_hours,n_steps,n_hours_out,n_features, dates, mean , std  = data_prep('MCC','2020-03-23', ['NO2','Volume'])
observed_p_NO2volLD, predicted_p_NO2volLD = evaluate_model(train_X_r, train_y, test_X_r, test_y,n_hours,n_steps, n_hours_out,n_features, dates, mean , std)

train_X_r, train_y, test_X_r, test_y, n_hours,n_steps,n_hours_out,n_features, dates, mean , std  = data_prep('MCC','2020-01-23', ['NO2','ws'])
observed_p_NO2ws, predicted_p_NO2ws = evaluate_model(train_X_r, train_y, test_X_r, test_y,n_hours,n_steps, n_hours_out,n_features, dates, mean , std)

train_X_r, train_y, test_X_r, test_y, n_hours,n_steps,n_hours_out,n_features, dates, mean , std  = data_prep('MCC','2020-03-23', ['NO2','ws'])
observed_p_NO2wsLD, predicted_p_NO2wsLD = evaluate_model(train_X_r, train_y, test_X_r, test_y,n_hours,n_steps, n_hours_out,n_features, dates, mean , std)

train_X_r, train_y, test_X_r, test_y, n_hours,n_steps,n_hours_out,n_features, dates, mean , std  = data_prep('MCC','2020-01-23', ['NO2','ws' ,'temp','wd_1', 'wd_2', 'wd_3', 'wd_4'])
observed_p_NO2met, predicted_p_NO2met = evaluate_model(train_X_r, train_y, test_X_r, test_y,n_hours,n_steps, n_hours_out,n_features, dates, mean , std)

train_X_r, train_y, test_X_r, test_y, n_hours,n_steps,n_hours_out,n_features, dates, mean , std  = data_prep('MCC','2020-03-23', ['NO2','ws' ,'temp','wd_1', 'wd_2', 'wd_3', 'wd_4'])
observed_p_NO2metLD, predicted_p_NO2metLD = evaluate_model(train_X_r, train_y, test_X_r, test_y,n_hours,n_steps, n_hours_out,n_features, dates, mean , std)

train_X_r, train_y, test_X_r, test_y, n_hours,n_steps,n_hours_out,n_features, dates, mean , std  = data_prep('MCC','2020-01-23', ['NO2','Volume' ,'ws' ,'temp','wd_1', 'wd_2', 'wd_3', 'wd_4'])
observed_p_NO2metvol, predicted_p_NO2metvol = evaluate_model(train_X_r, train_y, test_X_r, test_y,n_hours,n_steps, n_hours_out,n_features, dates, mean , std)

train_X_r, train_y, test_X_r, test_y, n_hours,n_steps,n_hours_out,n_features, dates, mean , std  = data_prep('MCC','2020-03-23', ['NO2','Volume' ,'ws' ,'temp','wd_1', 'wd_2', 'wd_3', 'wd_4'])
observed_p_NO2metvolLD, predicted_p_NO2metvolLD = evaluate_model(train_X_r, train_y, test_X_r, test_y,n_hours,n_steps, n_hours_out,n_features, dates, mean , std)

train_X_r, train_y, test_X_r, test_y, n_hours,n_steps,n_hours_out,n_features, dates, mean , std  = data_prep('MCC','2020-01-23', ['NO2', 'ws' ,'temp','wd_1', 'wd_2', 'wd_3', 'wd_4', 'Volume','Mon', 'Tue', 'Wed',
             'Thu', 'Fri', 'Sat', 'Sun'])
observed_p_allvars, predicted_p_allvars = evaluate_model(train_X_r, train_y, test_X_r, test_y,n_hours,n_steps, n_hours_out,n_features, dates, mean , std)
train_X_r, train_y, test_X_r, test_y, n_hours,n_steps,n_hours_out,n_features, dates, mean , std  = data_prep('MCC','2020-03-23', ['NO2', 'ws' ,'temp','wd_1', 'wd_2', 'wd_3', 'wd_4', 'Volume','Mon', 'Tue', 'Wed',
             'Thu', 'Fri', 'Sat', 'Sun'])
observed_p_allvarsLD, predicted_p_allvarsLD = evaluate_model(train_X_r, train_y, test_X_r, test_y,n_hours,n_steps, n_hours_out,n_features, dates, mean , std)

In [15]:
train_X_r, train_y, test_X_r, test_y, n_hours,n_steps,n_hours_out,n_features, dates, mean , std  = data_prep('sharston','2020-01-23',['NO2'])
observed_s_NO2, predicted_s_NO2 = evaluate_model(train_X_r, train_y, test_X_r, test_y,n_hours,n_steps, n_hours_out,n_features, dates, mean , std)

train_X_r, train_y, test_X_r, test_y, n_hours,n_steps,n_hours_out,n_features, dates, mean , std  = data_prep('sharston','2020-03-23', ['NO2'])
observed_s_NO2LD, predicted_s_NO2LD = evaluate_model(train_X_r, train_y, test_X_r, test_y,n_hours,n_steps, n_hours_out,n_features, dates, mean , std)

train_X_r, train_y, test_X_r, test_y, n_hours,n_steps,n_hours_out,n_features, dates, mean , std  = data_prep('sharston','2020-01-23', ['NO2','Volume'])
observed_s_NO2vol, predicted_s_NO2vol = evaluate_model(train_X_r, train_y, test_X_r, test_y,n_hours,n_steps, n_hours_out,n_features, dates, mean , std)

train_X_r, train_y, test_X_r, test_y, n_hours,n_steps,n_hours_out,n_features, dates, mean , std  = data_prep('sharston','2020-03-23', ['NO2','Volume'])
observed_s_NO2volLD, predicted_s_NO2volLD = evaluate_model(train_X_r, train_y, test_X_r, test_y,n_hours,n_steps, n_hours_out,n_features, dates, mean , std)

train_X_r, train_y, test_X_r, test_y, n_hours,n_steps,n_hours_out,n_features, dates, mean , std  = data_prep('sharston','2020-01-23', ['NO2','ws'])
observed_s_NO2ws, predicted_s_NO2ws = evaluate_model(train_X_r, train_y, test_X_r, test_y,n_hours,n_steps, n_hours_out,n_features, dates, mean , std)

train_X_r, train_y, test_X_r, test_y, n_hours,n_steps,n_hours_out,n_features, dates, mean , std  = data_prep('sharston','2020-03-23', ['NO2','ws'])
observed_s_NO2wsLD, predicted_s_NO2wsLD = evaluate_model(train_X_r, train_y, test_X_r, test_y,n_hours,n_steps, n_hours_out,n_features, dates, mean , std)

train_X_r, train_y, test_X_r, test_y, n_hours,n_steps,n_hours_out,n_features, dates, mean , std  = data_prep('sharston','2020-01-23', ['NO2','ws' ,'temp','wd_1', 'wd_2', 'wd_3', 'wd_4'])
observed_s_NO2met, predicted_s_NO2met = evaluate_model(train_X_r, train_y, test_X_r, test_y,n_hours,n_steps, n_hours_out,n_features, dates, mean , std)

train_X_r, train_y, test_X_r, test_y, n_hours,n_steps,n_hours_out,n_features, dates, mean , std  = data_prep('sharston','2020-03-23', ['NO2','ws' ,'temp','wd_1', 'wd_2', 'wd_3', 'wd_4'])
observed_s_NO2metLD, predicted_s_NO2metLD = evaluate_model(train_X_r, train_y, test_X_r, test_y,n_hours,n_steps, n_hours_out,n_features, dates, mean , std)

train_X_r, train_y, test_X_r, test_y, n_hours,n_steps,n_hours_out,n_features, dates, mean , std  = data_prep('sharston','2020-01-23', ['NO2','Volume' ,'ws' ,'temp','wd_1', 'wd_2', 'wd_3', 'wd_4'])
observed_s_NO2metvol, predicted_s_NO2metvol = evaluate_model(train_X_r, train_y, test_X_r, test_y,n_hours,n_steps, n_hours_out,n_features, dates, mean , std)

train_X_r, train_y, test_X_r, test_y, n_hours,n_steps,n_hours_out,n_features, dates, mean , std  = data_prep('sharston','2020-03-23', ['NO2','Volume' ,'ws' ,'temp','wd_1', 'wd_2', 'wd_3', 'wd_4'])
observed_s_NO2metvolLD, predicted_s_NO2metvolLD = evaluate_model(train_X_r, train_y, test_X_r, test_y,n_hours,n_steps, n_hours_out,n_features, dates, mean , std)

train_X_r, train_y, test_X_r, test_y, n_hours,n_steps,n_hours_out,n_features, dates, mean , std  = data_prep('sharston','2020-01-23', ['NO2', 'ws' ,'temp','wd_1', 'wd_2', 'wd_3', 'wd_4', 'Volume','Mon', 'Tue', 'Wed',
             'Thu', 'Fri', 'Sat', 'Sun'])
observed_s_allvars, predicted_s_allvars = evaluate_model(train_X_r, train_y, test_X_r, test_y,n_hours,n_steps, n_hours_out,n_features, dates, mean , std)
train_X_r, train_y, test_X_r, test_y, n_hours,n_steps,n_hours_out,n_features, dates, mean , std  = data_prep('sharston','2020-03-23', ['NO2', 'ws' ,'temp','wd_1', 'wd_2', 'wd_3', 'wd_4', 'Volume','Mon', 'Tue', 'Wed',
             'Thu', 'Fri', 'Sat', 'Sun'])
observed_s_allvarsLD, predicted_s_allvarsLD = evaluate_model(train_X_r, train_y, test_X_r, test_y,n_hours,n_steps, n_hours_out,n_features, dates, mean , std)

In [16]:
plt.figure(figsize=(30,30))
plt.subplot(6,2,1)
plot_trace (observed_p_NO2, predicted_p_NO2,dates_nld, 30, 'k', 'NO2')
plt.tick_params(
    axis='x',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom=False,      # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    labelbottom=False) # labels along the bottom edge are off
plt.subplot(6,2,2)
plot_trace (observed_p_NO2LD, predicted_p_NO2LD,dates_ld, 30, 'k', 'NO2')
plt.tick_params(
    axis='x',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom=False,      # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    labelbottom=False) # labels along the bottom edge are off
plt.subplot(6,2,3)
plot_trace (observed_p_NO2vol, predicted_p_NO2vol,dates_nld, 30, 'g', 'NO2 + Volume')
plt.tick_params(
    axis='x',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom=False,      # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    labelbottom=False) # labels along the bottom edge are off
plt.subplot(6,2,4)
plot_trace (observed_p_NO2volLD, predicted_p_NO2volLD,dates_ld, 30, 'g', 'NO2 + Volume')
plt.tick_params(
    axis='x',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom=False,      # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    labelbottom=False) # labels along the bottom edge are off
plt.subplot(6,2,5)
plot_trace (observed_p_NO2ws, predicted_p_NO2ws,dates_nld, 30, 'b', 'NO2 + ws')
plt.tick_params(
    axis='x',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom=False,      # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    labelbottom=False) # labels along the bottom edge are off
plt.subplot(6,2,6)
plot_trace (observed_p_NO2wsLD, predicted_p_NO2wsLD,dates_ld, 30, 'b', 'NO2 + ws')
plt.tick_params(
    axis='x',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom=False,      # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    labelbottom=False) # labels along the bottom edge are off
plt.subplot(6,2,7)
plot_trace (observed_p_NO2met, predicted_p_NO2met,dates_nld, 30, 'orange', 'NO2 + Met')
plt.tick_params(
    axis='x',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom=False,      # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    labelbottom=False) # labels along the bottom edge are off
plt.subplot(6,2,8)
plot_trace (observed_p_NO2metLD, predicted_p_NO2metLD,dates_ld, 30, 'orange', 'NO2 + Met')
plt.tick_params(
    axis='x',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom=False,      # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    labelbottom=False) # labels along the bottom edge are off
plt.subplot(6,2,9)
plot_trace (observed_p_NO2metvol, predicted_p_NO2metvol,dates_nld, 30, 'c', 'NO2+Met+Vol')
plt.tick_params(
    axis='x',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom=False,      # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    labelbottom=False) # labels along the bottom edge are off
plt.subplot(6,2,10)
plot_trace (observed_p_NO2metvolLD, predicted_p_NO2metvolLD,dates_ld, 30, 'c', 'NO2+Met+Vol')
plt.tick_params(
    axis='x',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom=False,      # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    labelbottom=False) # labels along the bottom edge are off
plt.subplot(6,2,11)
plot_trace (observed_p_allvars, predicted_p_allvars,dates_nld, 30, 'y', 'All Variables')
plt.subplot(6,2,12)
plot_trace (observed_p_allvarsLD, predicted_p_allvarsLD,dates_ld, 30, 'y', 'All Variables')
plt.tight_layout()
plt.savefig('./FiguresLSTM/Piccadilly_CNNLSTM.png')

In [17]:
plt.figure(figsize=(30,30))
plt.subplot(6,2,1)
plot_trace (observed_s_NO2, predicted_s_NO2, dates_nld, 30, 'k', 'NO2')
plt.tick_params(
    axis='x',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom=False,      # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    labelbottom=False) # labels along the bottom edge are off
plt.subplot(6,2,2)
plot_trace (observed_s_NO2LD, predicted_s_NO2LD, dates_ld, 30, 'k', 'NO2')
plt.tick_params(
    axis='x',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom=False,      # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    labelbottom=False) # labels along the bottom edge are off
plt.subplot(6,2,3)
plot_trace (observed_s_NO2vol, predicted_s_NO2vol, dates_nld, 30, 'g', 'NO2 + Volume')
plt.tick_params(
    axis='x',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom=False,      # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    labelbottom=False) # labels along the bottom edge are off
plt.subplot(6,2,4)
plot_trace (observed_s_NO2volLD, predicted_s_NO2volLD, dates_ld, 30, 'g', 'NO2 + Volume')
plt.tick_params(
    axis='x',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom=False,      # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    labelbottom=False) # labels along the bottom edge are off
plt.subplot(6,2,5)
plot_trace (observed_s_NO2ws, predicted_s_NO2ws, dates_nld, 30, 'b', 'NO2 + ws')
plt.tick_params(
    axis='x',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom=False,      # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    labelbottom=False) # labels along the bottom edge are off
plt.subplot(6,2,6)
plot_trace (observed_s_NO2wsLD, predicted_s_NO2wsLD, dates_ld, 30, 'b', 'NO2 + ws')
plt.tick_params(
    axis='x',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom=False,      # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    labelbottom=False) # labels along the bottom edge are off
plt.subplot(6,2,7)
plot_trace (observed_s_NO2met, predicted_s_NO2met, dates_nld, 30, 'orange', 'NO2 + Met')
plt.tick_params(
    axis='x',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom=False,      # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    labelbottom=False) # labels along the bottom edge are off
plt.subplot(6,2,8)
plot_trace (observed_s_NO2metLD, predicted_s_NO2metLD, dates_ld, 30, 'orange', 'NO2 + Met')
plt.tick_params(
    axis='x',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom=False,      # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    labelbottom=False) # labels along the bottom edge are off
plt.subplot(6,2,9)
plot_trace (observed_s_NO2metvol, predicted_s_NO2metvol, dates_nld, 30, 'c', 'NO2+Met+Vol')
plt.tick_params(
    axis='x',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom=False,      # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    labelbottom=False) # labels along the bottom edge are off
plt.subplot(6,2,10)
plot_trace (observed_s_NO2metvolLD, predicted_s_NO2metvolLD, dates_ld, 30, 'c', 'NO2+Met+Vol')
plt.tick_params(
    axis='x',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom=False,      # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    labelbottom=False) # labels along the bottom edge are off
plt.subplot(6,2,11)
plot_trace (observed_s_allvars, predicted_s_allvars, dates_nld, 30, 'y', 'All Variables')
plt.subplot(6,2,12)
plot_trace (observed_s_allvarsLD, predicted_s_allvarsLD, dates_ld, 30, 'y', 'All Variables')
plt.tight_layout()
plt.savefig('./FiguresLSTM/sharston_CNNLSTM.png')